In [23]:
import numpy as np
import gym
import tensorflow as tf
#from tensorflow.keras.layers import Dense, Flatten
from keras.layers import Dense, Flatten
#from tensorflow.keras.models import Sequential
from keras.models import Sequential
#from tensorflow.keras.optimizers import RMSprop
from keras import optimizers

from rl.agents.dqn import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy,LinearAnnealedPolicy, BoltzmannQPolicy, GreedyQPolicy, MaxBoltzmannQPolicy, BoltzmannGumbelQPolicy, SoftmaxPolicy
from rl.core import Processor

In [24]:
def create_agent(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [25]:
class TaxiProcessor(Processor):
    def process_observation(self, observation):
        one_hot = np.zeros(500)
        one_hot[observation] = 1
        return one_hot

    def process_reward(self, reward):
        return reward

In [26]:
env = gym.make("Taxi-v3")
print("No of state = ", env.observation_space.n)
print("No of actions =", env.action_space.n)
model = create_agent(500, env.action_space.n)
model.summary()

No of state =  500
No of actions = 6
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 500)               0         
                                                                 
 dense_12 (Dense)            (None, 24)                12024     
                                                                 
 dense_13 (Dense)            (None, 24)                600       
                                                                 
 dense_14 (Dense)            (None, 24)                600       
                                                                 
 dense_15 (Dense)            (None, 6)                 150       
                                                                 
Total params: 13,374
Trainable params: 13,374
Non-trainable params: 0
_________________________________________________________________


In [27]:
memory = SequentialMemory(limit=800000, window_length=1)
processor = TaxiProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)

#policy = LinearAnnealedPolicy(MaxBoltzmannQPolicy(), attr='eps', value_max=1., value_min=.05, value_test=.005, nb_steps=10000)
dqn = DQNAgent(model=model, nb_actions=env.action_space.n, processor=processor, memory=memory, nb_steps_warmup=1000, gamma=0.99, policy=policy, enable_double_dqn=True, target_model_update=5e-3)



In [28]:
optimizer = tf.keras.optimizers.legacy.RMSprop(lr=1e-3)
optimizer1 = optimizers.Adam(lr=1e-3)
dqn.compile(optimizer1, metrics=['mae'])
dqn.fit(env, nb_steps=250000, visualize=False, verbose=1)

Training for 250000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 76s 8ms/step - reward: -2.4883
50 episodes - episode_reward: -496.660 [-857.000, -209.000] - loss: 0.655 - mae: 12.801 - mean_q: -11.858 - mean_eps: 0.478 - prob: 1.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 82s 8ms/step - reward: -1.1563
50 episodes - episode_reward: -230.260 [-353.000, -165.000] - loss: 1.355 - mae: 21.519 - mean_q: -21.993 - mean_eps: 0.050 - prob: 1.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: -1.1668
56 episodes - episode_reward: -209.482 [-722.000, 12.000] - loss: 2.774 - mae: 29.366 - mean_q: -31.263 - mean_eps: 0.050 - prob: 1.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 86s 9ms/step - reward: -1.1101
65 episodes - episode_reward: -170.969 [-341.000, 12.000] - loss: 2.860 - mae: 28.705 - mean_q: -30.077 - m

In [29]:
env.reset()
res = dqn.test(env, nb_episodes=100, visualize=False)


Testing for 100 episodes ...
Episode 1: reward: 12.000, steps: 9
Episode 2: reward: 7.000, steps: 14
Episode 3: reward: 4.000, steps: 17
Episode 4: reward: 5.000, steps: 16
Episode 5: reward: 8.000, steps: 13
Episode 6: reward: 10.000, steps: 11
Episode 7: reward: 14.000, steps: 7
Episode 8: reward: 6.000, steps: 15
Episode 9: reward: 7.000, steps: 14
Episode 10: reward: 11.000, steps: 10
Episode 11: reward: 10.000, steps: 11
Episode 12: reward: 10.000, steps: 11
Episode 13: reward: 3.000, steps: 18
Episode 14: reward: 8.000, steps: 13
Episode 15: reward: 4.000, steps: 17
Episode 16: reward: 13.000, steps: 8
Episode 17: reward: 10.000, steps: 11
Episode 18: reward: 7.000, steps: 14
Episode 19: reward: 6.000, steps: 15
Episode 20: reward: 10.000, steps: 11
Episode 21: reward: 4.000, steps: 17
Episode 22: reward: 4.000, steps: 17
Episode 23: reward: 5.000, steps: 16
Episode 24: reward: 10.000, steps: 11
Episode 25: reward: 11.000, steps: 10
Episode 26: reward: 8.000, steps: 13
Episode 27

In [30]:
res.history
print(np.average(res.history['episode_reward']))

7.74
